<a href="https://colab.research.google.com/github/azdinelaaouissi/data_cars/blob/main/cars_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import  requests
import  re
import pandas as  pd
from bs4  import   BeautifulSoup
import csv
import  json

In [51]:
def  car_info(url):
  response=requests.get(url)
  response.encoding=response.apparent_encoding
  if  response.status_code==200:
    soup = BeautifulSoup(response.text, 'html.parser')
    script_content=soup.find("div",id='screenL').find("script").string
    if script_content:
      pattern = r'tagInfos\s*=\s*(\{.*?\});'
      match = re.search(pattern, script_content)
      print("====================")
      data=None
      if match:
        json_data = match.group(1)
        data_dict = json.loads(json_data)

        brand=data_dict.get("ft_brand",None)
        model=data_dict.get("ft_model",None)
        version=data_dict.get("ft_version",None)
        carburant=data_dict.get("ft_fuel",None)
        ville=data_dict.get("ft_city",None)
        annee=data_dict.get("ft_year",None)
        Kilometrage=None
        Puissance_fiscale=None
        prix=None
        if  soup.find("div",class_="color_primary text_bold price-block"):
          prix=soup.find("div",class_="color_primary text_bold price-block").text.strip()

        if   soup.find('span', string='Puissance fiscale'):
          Puissance_fiscale=soup.find('span', string='Puissance fiscale').find_next_sibling('span', class_='text_bold').text.strip()

        if soup.find('span', string='Kilométrage'):
          Kilometrage=soup.find('span', string='Kilométrage').find_next_sibling("span",class_="text_bold").text.strip()
        # print(Puissance_fiscale)
        data={
            "brand":brand,
            "model":model,
            "version":version,
            "carburant":carburant,
            "ville":ville,
            "annee":annee,
            "Kilometrage":Kilometrage,
            "puissance_fiscale":Puissance_fiscale,
            "prix":prix
        }
      return  data
    else:
      print("Données JSON non trouvées dans le script")
      return  None;
  else:
    print("error",response.status_code)
    return None;

In [62]:
data_cars=[]
for  i in  range(0,6):
    url="https://www.moteur.ma/fr/voiture/achat-voiture-occasion/"+str(i*30)
    response=requests.get(url)
    response.encoding="utf-8"
    if response.status_code==200:
      html=response.text
      soup=BeautifulSoup(html,"html.parser")
      annonces=soup.findAll("div",class_="content-inner")
      for  annonce in  annonces:
        a_car=annonce.findAll("div", class_="meta")[-1].find("ul").findAll("li")[-1].find("a").get("href")
        #  print(a_car)
        d=car_info(a_car)
        if d:
         data_cars.append(d)
    else:
      print(f"erooor==>{response.status_code}")

df=pd.DataFrame(data_cars)
print(df)
df.to_csv("cars_df.csv",index=False)

Données JSON non trouvées dans le script
Données JSON non trouvées dans le script
Données JSON non trouvées dans le script
Données JSON non trouvées dans le script
          brand       model          version carburant       ville annee  \
0         skoda      superb                     diesel  casablanca  2019   
1      mercedes    classe c             c220    diesel       nador  2008   
2      mercedes    classe e                     diesel       nador  1994   
3      mercedes         250                     diesel       nador  1995   
4      mercedes   classe ml                     diesel       nador  2006   
..          ...         ...              ...       ...         ...   ...   
171         bmw     serie 3           lounge    diesel  casablanca  2020   
172  volkswagen      tiguan         2.0l tdi    diesel  casablanca  2016   
173        mini  countryman               sd    diesel  casablanca  2017   
174      jaguar          xf           luxury    diesel  casablanca  2019   


In [63]:
df.shape

(176, 9)